<h1> Collecting the tweet entities</h1>

In [1]:
pip install git+https://github.com/tweepy/tweepy.git

  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-l710k1_v
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-l710k1_v
  Resolved https://github.com/tweepy/tweepy.git to commit f32d12dbddbd877470446657812a10a04292d0c9
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
import tweepy


In [ ]:
bearer_token='Your API'
client = tweepy.Client(bearer_token)

In [ ]:
q = "(#malware) OR (#ransomware) OR (#botnet) OR (#androidtrojan) OR (#bankingtrojan) OR (#windowstrojan) OR (#remoteaccesstrojan) OR (#rattrojan) OR (#linuxtrojan) OR (#adware)	OR (#spyware)	OR (#keylogger)	OR (#rootkit)	OR (#bootkit)	OR (#rigek)	OR (#exploitkit) OR (#honeypot)	OR (#ioc)	OR (#malspam)	OR (#crypto-locker)	OR (#cryptolocker) OR (#crypto-ransomware) OR (#opendir) OR (app.any.run)	OR (pastebin.com)	OR (#c2) OR (#md5) OR (#sha256)	OR (otx.alienvault.com/pulse)	OR (urlhaus.abuse.ch)	OR (hybrid-analysis.com) OR (urlscan.io) OR (#spearphishing) OR (#spear-phishing)	OR (#phishing) OR (#phishingkit) OR (#cyberattack) OR (#cve) OR (#rootkit) OR (#sqlinjection)	OR (#keylogging) OR (#0day) OR (#keylogger) OR (#passwordcracker)	OR (#metasploit) OR (#reduc) OR (#zombie) OR (#C&C) OR (#emotet) OR (#cobaltstrike) OR (#scam) OR (#redline) OR (#formbook) OR (#lokibot) OR (#raccoon) lang:en"

In [ ]:
len(q)

896

In [ ]:
starttime="2021-11-01T00:00:00-00:00"
endtime="2021-11-10T11:59:59-00:00"

In [ ]:
#No need to run second time.
tweets = client.search_all_tweets(query = q,
                                  max_results = 500,
                                  start_time = starttime,
                                  end_time = endtime,
                                  tweet_fields = ['attachments', 'author_id', 'conversation_id', 'created_at', 'entities', 'geo', 'id', 'in_reply_to_user_id', 'lang', 'public_metrics', 'possibly_sensitive', 'referenced_tweets', 'reply_settings', 'source', 'text', 'withheld'],
                                  media_fields = ['duration_ms', 'height', 'media_key', 'preview_image_url', 'type', 'url', 'width', 'public_metrics', 'alt_text'],
                                  place_fields= ['contained_within', 'country', 'country_code', 'full_name', 'geo', 'id', 'name', 'place_type'],
                                  poll_fields=['duration_minutes', 'end_datetime', 'id', 'options', 'voting_status'],
                                  user_fields=['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id', 'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'withheld'],
                                  expansions=['attachments.poll_ids', 'attachments.media_keys', 'author_id', 'entities.mentions.username', 'geo.place_id', 'in_reply_to_user_id', 'referenced_tweets.id', 'referenced_tweets.id.author_id'])

In [ ]:
tweets.data

In [ ]:
def user_entity_value(dicts,key1,key2,key3):
  user_value = []
  if dicts is not None:
    for i in dicts:
      if i == key1:
        for j in dicts[i]:
          if j == key2:
            for k in range(0,len(dicts[i][j])):
              if key3 in dicts[i][j][k]:
                user_value.append(dicts[i][j][k][key3])
    if user_value:
      return user_value
    else:
      return None
  else:
    return None

In [ ]:
def tweet_entity_value(dicts,key1,key2):
  tweet_value=[]
  if dicts is not None:
    for i in dicts:
      if i == key1:
        for j in range(0,len(dicts[i])):
          if key2 in dicts[i][j]:
            tweet_value.append(dicts[i][j][key2])
    if tweet_value:
      return tweet_value
    else:
      return None
  else:
    return None

In [ ]:
def tweet_referenced_id(rlist):
  if rlist is not None:
    ref_id=str(str(rlist).split('=')[1]).split()
    return ref_id[0]
  else:
    return None

In [ ]:
def tweet_referenced_type(rlist):
  if rlist is not None:
    ref_type=str(str(rlist).split('=')[2]).split(']')
    return ref_type[0]
  else:
    return None

In [ ]:
def UserInfo():
    for u in tweets.includes['users']:
      user_info={
        'ID':u.id,
        'User Name':u.username,
        'Screen Name':u.name,
        'Creation Date':u.created_at,
        'Location':u.location,
        'Description':u.description,
        'Followers Count':u.public_metrics['followers_count'],
        'Following Count':u.public_metrics['following_count'],
        'Listed Count':u.public_metrics['listed_count'],
        'Tweet Count':u.public_metrics['tweet_count'],
        'Profile Image URL':u.profile_image_url,
        'URL':u.url,
        'Protected':u.protected,
        'Verified':u.verified,
        'Pinned Tweet ID':u.pinned_tweet_id,
        'Entities': u.entities,
        'Username mentioned in Entity':user_entity_value(u.entities,'description','mentions','username'),
        'URL mentioned in Entity':user_entity_value(u.entities,'url','urls','url'),
        'Expanded URL mentioned in Entity':user_entity_value(u.entities,'url','urls','expanded_url'),
        'Display URL mentioned in Entity':user_entity_value(u.entities,'url','urls','display_url'),
        'Hashtags mentioned in Entity': user_entity_value(u.entities,'description','hashtags','tag')
        }
      user_info_ls.append(user_info)



In [ ]:
def return_name(author_id,name):
  for u in tweets.includes['users']:
    if u.id==author_id:
      if name=='username':
        return u.username
      elif name=='name':
        return u.name


In [ ]:
def TweetValues():
  if tweets.data is not None:
    # iterate over each tweet and corresponding user details
    for tweet in tweets.data:
      tweet_info = {
        'Author ID':tweet.author_id,
        'User Name': return_name(tweet.author_id,'username'),
        'Screen Name':return_name(tweet.author_id,'name'),
        'Tweet ID': tweet.id,
        'Created At': tweet.created_at,
        'Text': tweet.text,
        'Like Count':tweet.public_metrics['like_count'],
        'Quote Count': tweet.public_metrics['quote_count'],
        'Reply Count': tweet.public_metrics['reply_count'],
        'Retweet Count': tweet.public_metrics['retweet_count'],
        'Entities':tweet.entities,
        'Expanded URL': tweet_entity_value(tweet.entities,'urls','expanded_url'),
        'Hashtag':tweet_entity_value(tweet.entities,'hashtags','tag'),
        'Mentioned Username':tweet_entity_value(tweet.entities,'mentions','username'),
        'Annotated Entity':tweet_entity_value(tweet.entities,'annotations','normalized_text'),
        'Annotation Probability':tweet_entity_value(tweet.entities,'annotations','probability'),
        'Annotation Type':tweet_entity_value(tweet.entities,'annotations','type'),
        'Conversation ID': tweet.conversation_id,
        'Referenced Tweets': tweet.referenced_tweets,
        'ID of Referenced Tweets': tweet_referenced_id(tweet.referenced_tweets),
        'Type of Referenced Tweets': tweet_referenced_type(tweet.referenced_tweets),
        'Attachments': tweet.attachments,
        'Location tagged by the user': tweet.geo,
        'Language':tweet.lang,
        'Sensitive URL': tweet.possibly_sensitive,
        'Reply Settings': tweet.reply_settings,
        'Source': tweet.source
        }
      if tweet_info['Language']=='en':
        tweet_info_ls.append(tweet_info)

In [ ]:
#Second time skip this code
# create a list of records
tweet_info_ls = []
user_info_ls = []
TweetValues()
UserInfo()
#tweet_df1 = pd.DataFrame(tweet_info_ls)
#user_df1 = pd.DataFrame(user_info_ls)

In [ ]:
while True:
  if 'next_token' in tweets.meta:
    tweets = client.search_all_tweets(query = q,
                                      max_results = 500,
                                      start_time = starttime,
                                      end_time = endtime,
                                      tweet_fields = ['attachments', 'author_id', 'conversation_id', 'created_at', 'entities', 'geo', 'id', 'in_reply_to_user_id', 'lang', 'public_metrics', 'possibly_sensitive', 'referenced_tweets', 'reply_settings', 'source', 'text', 'withheld'],
                                      media_fields = ['duration_ms', 'height', 'media_key', 'preview_image_url', 'type', 'url', 'width', 'public_metrics', 'alt_text'],
                                      place_fields= ['contained_within', 'country', 'country_code', 'full_name', 'geo', 'id', 'name', 'place_type'],
                                      poll_fields=['duration_minutes', 'end_datetime', 'id', 'options', 'voting_status'],
                                      user_fields=['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id', 'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'withheld'],
                                      expansions=['attachments.poll_ids', 'attachments.media_keys', 'author_id', 'entities.mentions.username', 'geo.place_id', 'in_reply_to_user_id', 'referenced_tweets.id', 'referenced_tweets.id.author_id'],
                                      next_token = tweets.meta['next_token'])
    TweetValues()
    UserInfo()
  else:
    break
tweet_df1 = pd.DataFrame(tweet_info_ls)
user_df1 = pd.DataFrame(user_info_ls)


In [ ]:
'''
tweet_info_ls = []
user_info_ls = []
last_date=tweet_df1['Created At'][tweet_df1.index[-1]]
for i in range(0,50):
  tweets = client.search_all_tweets(query = q,max_results = 500, start_time = "2021-06-01T00:00:00-00:00", end_time = last_date, tweet_fields = ['attachments', 'author_id', 'conversation_id', 'created_at', 'entities', 'geo', 'id', 'in_reply_to_user_id', 'lang', 'public_metrics', 'possibly_sensitive', 'referenced_tweets', 'reply_settings', 'source', 'text', 'withheld'],media_fields = ['duration_ms', 'height', 'media_key', 'preview_image_url', 'type', 'url', 'width', 'public_metrics', 'alt_text'],place_fields= ['contained_within', 'country', 'country_code', 'full_name', 'geo', 'id', 'name', 'place_type'],poll_fields=['duration_minutes', 'end_datetime', 'id', 'options', 'voting_status'],user_fields=['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id', 'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'withheld'],expansions=['attachments.poll_ids', 'attachments.media_keys', 'author_id', 'entities.mentions.username', 'geo.place_id', 'in_reply_to_user_id', 'referenced_tweets.id', 'referenced_tweets.id.author_id'])
  TweetValues()
  UserInfo()
  # create dataframe from the extracted records
  tweet_df2 = pd.DataFrame(tweet_info_ls)
  user_df2=pd.DataFrame(user_info_ls)
  tweet_df1 = tweet_df1.append(tweet_df2, ignore_index = True)
  user_df1 = user_df1.append(user_df2, ignore_index = True)
  last_date=tweet_df2['Created At'][tweet_df2.index[-1]]
'''

In [ ]:
last_date=tweet_df1['Created At'][tweet_df1.index[-1]]
last_date

Timestamp('2021-10-01 00:00:03+0000', tz='UTC')

In [ ]:
len(tweet_df1)

71012

In [ ]:
user_df1.to_csv("/content/drive/MyDrive/Final_Dataset/October/October_2021_11_20_user.csv")
tweet_df1.to_csv("/content/drive/MyDrive/Final_Dataset/October/October_2021_11_20_tweet.csv")